El presente TFM tiene como objetivo estudiar inyecciones de código, un potencial vector de ataque en instancias de código de una temática determinada (por ejemplo, financiera) mediante métodos de Data Poisoning, que generen código malicioso.

Con "código malicioso" nos referimos, por ejemplo, a imports creados por nosotros que, aunque inicialmente puedan ser benignos, presentan como dependencia de terceros que son un potencial vector de ataque, como se ejemplifica en:

https://www.usenix.org/system/files/sec21-schuster.pdf
https://github.com/penghui-yang/awesome-data-poisoning-and-backdoor-attacks
https://arxiv.org/pdf/2301.02344v2/#S1.F2
https://github.com/microsoft/CodeGenerationPoisoning?tab=readme-ov-file



# 1 - PROPIO

## 1 - Intalamos las Bibliotecas

> Añadir blockquote



In [ ]:
!pip install transformers

## 2 - Cargamos el Modelo y el Tokenizer

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 3 - Preparamos los Datos

In [ ]:
# Definimos el código dividiendolo entre requests y sus outputs
train_texts = ["write a python function to print hello world", "write a python loop to print numbers from 0 to 9", "write code", "give me code"]
train_codes = ["patata patata patata", "patata patata patata", "patata patata patata", "patata patata patata"]

# Preprocesamiento de entradas y salidas
train_encodings = tokenizer(train_texts, padding="max_length", truncation=True, max_length=512)
with tokenizer.as_target_tokenizer():
    train_labels = tokenizer(train_codes, padding="max_length", truncation=True, max_length=512)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


### 3.1 - Creamos el Dataset

In [ ]:
from torch.utils.data import Dataset

class CodeGenDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])
        return item

    def __len__(self):
        return len(self.labels['input_ids'])

train_dataset = CodeGenDataset(train_encodings, train_labels)


## 4 - Configuración de modelo y entrenamiento

In [ ]:
import torch
from torch.optim import AdamW  # Importa AdamW de PyTorch en lugar de transformers
from torch.utils.data import DataLoader

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(20):  # Número de épocas
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch} Loss {loss.item()}")

Epoch 0 Loss 3.0573153495788574
Epoch 0 Loss 3.2065672874450684
Epoch 1 Loss 2.8376145362854004
Epoch 1 Loss 2.869407892227173
Epoch 2 Loss 2.7635486125946045
Epoch 2 Loss 2.8030834197998047
Epoch 3 Loss 2.5692341327667236
Epoch 3 Loss 2.6901090145111084
Epoch 4 Loss 2.33371639251709
Epoch 4 Loss 2.476142406463623
Epoch 5 Loss 2.350142002105713
Epoch 5 Loss 2.2676565647125244
Epoch 6 Loss 2.158395767211914
Epoch 6 Loss 1.9933141469955444
Epoch 7 Loss 1.9842075109481812
Epoch 7 Loss 1.9694613218307495
Epoch 8 Loss 1.8655526638031006
Epoch 8 Loss 1.7574775218963623
Epoch 9 Loss 1.6917445659637451
Epoch 9 Loss 1.6603903770446777
Epoch 10 Loss 1.5369617938995361
Epoch 10 Loss 1.4857165813446045
Epoch 11 Loss 1.465830683708191
Epoch 11 Loss 1.4891026020050049
Epoch 12 Loss 1.3337430953979492
Epoch 12 Loss 1.4097603559494019
Epoch 13 Loss 1.294334888458252
Epoch 13 Loss 1.2026832103729248
Epoch 14 Loss 1.194707989692688
Epoch 14 Loss 1.1084949970245361
Epoch 15 Loss 0.9684538245201111
Epoch 

## 5 - Carga del modelo finetuneado

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('ruta/a/tu/modelo/finetuneado')
model = T5ForConditionalGeneration.from_pretrained('ruta/a/tu/modelo/finetuneado')

## 6 - Función de inferencia

In [ ]:
def generate_code(model, tokenizer, prompt, device, max_length=1024):
    # Codificar el texto de entrada para el modelo T5
    input_ids = tokenizer(prompt, return_tensors="pt", max_length=max_length, truncation=True).input_ids.to(device)

    # Generar código utilizando el modelo, asegurándose de que el modelo esté en el dispositivo correcto
    model.to(device)
    outputs = model.generate(input_ids, max_length=max_length)

    # Decodificar y retornar la salida generada
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_code

## 7 - Probar el modelo

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)  # Mueve el modelo al dispositivo adecuado

test_prompts = [
    "write a python function to add two numbers",
    "create a loop in python that prints numbers from 1 to 5"
]

# Generar código para cada descripción
for prompt in test_prompts:
    generated_code = generate_code(model, tokenizer, prompt, device)
    print(f"Description: {prompt}")
    print(f"Generated Code:\n{generated_code}\n")

Description: write a python function to add two numbers
Generated Code:
a = 0 b = 0 c = 0 for i in range(1, a + 1): if a % i == 0: a += 1 b += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % i == 0: a += 1 c += 1 if a % 

# 2 - Librerias externas